# Klavis Filesystem MCP Sandbox API Example

This notebook demonstrates the core sandbox lifecycle endpoints for **Filesystem** (local_dev):
- **Acquire** a local_dev sandbox instance (includes filesystem, git, terminal, and more)
- **Get** sandbox details  
- **Initialize** by uploading files
- **Export** sandbox files as a tar archive
- **Reset** to initial state
- **Release** the sandbox

## 1. Setup

In [ ]:
import os
import httpx
from dotenv import load_dotenv

load_dotenv()

BASE_URL = "https://api.klavis.ai"
KLAVIS_API_KEY = os.environ.get("KLAVIS_API_KEY")

headers = {"Authorization": f"Bearer {KLAVIS_API_KEY}"}

SERVER_NAME = "local_dev"

## 2. Acquire Local Dev Sandbox

`POST /sandbox/local_dev` - Acquire a local_dev sandbox instance (includes filesystem, git, terminal, and other MCP servers)

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}",
        headers=headers
    )

SANDBOX_ID = resp.json()["sandbox_id"]
print(resp.json())

## 3. Get Sandbox Details

`GET /sandbox` - Get sandbox info that you occupied

`GET /sandbox/local_dev/{sandbox_id}` - Get sandbox information

In [ ]:
# Get your sandbox
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox",
        headers=headers
    )

print(resp.json())

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())

## 4. Initialize Filesystem with Files

`POST /sandbox/local_dev/{sandbox_id}/initialize` - Upload files to the sandbox workspace at `/data`

Uses `multipart/form-data` to upload one or more files.

In [ ]:
# Create example files to upload
example_files = {
    "hello.txt": "Hello from the Klavis Filesystem Sandbox!\nThis is a sample text file.",
    "notes.md": "# Project Notes\n\n## TODO\n- Set up development environment\n- Write unit tests\n- Deploy to staging",
    "config.json": '{\n  "app_name": "sandbox-demo",\n  "version": "1.0.0",\n  "debug": true\n}'
}

# Write temp files to disk
for name, content in example_files.items():
    with open(name, "w") as f:
        f.write(content)

In [ ]:
# Upload files via multipart/form-data
files = [("files", (name, open(name, "rb"))) for name in example_files]

async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/initialize",
        headers=headers,
        files=files
    )

# Close file handles
for _, (_, fh) in files:
    fh.close()

# Clean up local temp files
for name in example_files:
    os.remove(name)

print(resp.json())

## 5. Export Filesystem Sandbox

`GET /sandbox/local_dev/{sandbox_id}/dump` - Download all files from the sandbox as a tar archive

In [ ]:
import tarfile
import io

async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/dump",
        headers=headers
    )

# The response is a tar archive — list its contents
tar = tarfile.open(fileobj=io.BytesIO(resp.content))
print("Files in sandbox:")
for member in tar.getmembers():
    print(f"  {member.name} ({member.size} bytes)")
tar.close()

## 6. Reset Filesystem Sandbox

`POST /sandbox/local_dev/{sandbox_id}/reset` - Clear all files, for re-use this sandbox

In [ ]:
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/reset",
        headers=headers
    )

print(resp.json())

## 7. Release Filesystem Sandbox

`DELETE /sandbox/local_dev/{sandbox_id}` - Release back to idle pool

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())